In [1]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f
from isodate import parse_duration
import pyspark.sql.types as t
from typing import Tuple
import os

In [2]:
@f.udf(returnType=t.IntegerType())
def converter_segundos(tempo: str) -> int:
    """Converte para o total de seguntos

    Args:
        tempo (str): tempo

    Returns:
        int: total segundos
    """
    duracao = parse_duration(tempo)
    total_minutes = duracao.total_seconds() 
   
    return int(total_minutes)


In [3]:
spark = SparkSession.builder.appName('tratamento_camada_ouro').getOrCreate()
spark

23/11/01 00:08:26 WARN Utils: Your hostname, rodrigo-Nitro-AN515-54 resolves to a loopback address: 127.0.1.1; using 192.168.0.112 instead (on interface wlp8s0)
23/11/01 00:08:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/01 00:08:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
def carregar_dataframe_prata(spark_session : SparkSession, assunto: str, metrica: str,):
    df_cities_skylines = spark_session \
        .read \
            .parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/prata/{assunto}/extracao_data_*/{metrica}/*.parquet')
    return df_cities_skylines

In [4]:
def criar_particao_ouro(dataframe: DataFrame, assunto_pesquisa: str, metrica: str,  nome_arquivo: str, colunas_particao: Tuple = None):
    caminho_datalake_ouro = '/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/'
    if colunas_particao == None:
        dataframe.write. \
            parquet(os.path.join(caminho_datalake_ouro, assunto_pesquisa, metrica, nome_arquivo))
    else:
        dataframe.write. \
            partitionBy(*colunas_particao). \
            parquet(os.path.join(caminho_datalake_ouro, assunto_pesquisa, metrica, nome_arquivo))


In [6]:
def transformacao_dataframe(dataframe_transform: DataFrame):
    dataframe_transform = dataframe_transform.select(
        'data_extracao',
        f.explode('DATA_PUBLICACAO').alias('DATA_PUBLICACAO'),
        'ID_CANAL',
        'NM_CANAL',
        'ID_CATEGORIA',
        'ID_VIDEO',
        'TITULO_VIDEO',
        'DESCRICAO',
        'TAGS',
        'DURACAO_VIDEOS',
        'TOTAL_VISUALIZACOES',
        'TOTAL_LIKES',
        'TOTAL_FAVORITOS',
        'TOTAL_COMENTARIOS',
        
     ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                f.explode('ID_CANAL').alias('ID_CANAL'),
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                f.explode('NM_CANAL').alias('NM_CANAL'), 
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                f.explode('ID_CATEGORIA').alias('ID_CATEGORIA'),
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                f.explode('ID_VIDEO').alias('ID_VIDEO'),
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                f.explode('TITULO_VIDEO').alias('TITULO_VIDEO'),
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                'ID_VIDEO',
                f.explode('DESCRICAO').alias('DESCRICAO'),
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                f.explode('TAGS').alias('TAGS'),
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                f.explode('DURACAO_VIDEOS').alias('DURACAO_VIDEOS'),
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                f.explode('TOTAL_VISUALIZACOES').alias('TOTAL_VISUALIZACOES'),
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                f.explode('TOTAL_LIKES').alias('TOTAL_LIKES'),
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                f.explode('TOTAL_FAVORITOS').alias('TOTAL_FAVORITOS'),
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                f.explode('TOTAL_COMENTARIOS').alias('TOTAL_COMENTARIOS'),
        )
    dataframe_transform  = dataframe_transform.na.drop('all')
    dataframe_transform = dataframe_transform.withColumn("data_hora_extracao", f.col('data_extracao'))
    dataframe_transform = dataframe_transform.withColumn("data_extracao", f.date_format(f.unix_timestamp("data_extracao", "yyyy-MM-dd HH:mm:ss").cast("timestamp"), "yyyy-MM-dd"))
    dataframe_transform = dataframe_transform.withColumn('DURACAO_VIDEO_SEGUNDOS', converter_segundos(f.col('DURACAO_VIDEOS')))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_VISUALIZACOES', f.col('TOTAL_VISUALIZACOES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_LIKES', f.col('TOTAL_LIKES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_FAVORITOS', f.col('TOTAL_FAVORITOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_COMENTARIOS', f.col('TOTAL_COMENTARIOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_CARACTERE_VIDEO', f.length('TITULO_VIDEO'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_TAGS', f.size('TAGS'))
    return dataframe_transform
  

In [10]:
@f.udf(returnType=t.StringType())
def traduzir_dia_da_semana(dia):
    traducao = {
        'Sunday': 'Domingo',
        'Monday': 'Segunda-feira',
        'Tuesday': 'Terça-feira',
        'Wednesday': 'Quarta-feira',
        'Thursday': 'Quinta-feira',
        'Friday': 'Sexta-feira',
        'Saturday': 'Sábado'
    }
    return traducao.get(dia, dia)

In [11]:
@f.udf(returnType=t.IntegerType())
def indice_semana(dia):
    traducao = {
        'Domingo': 1,
        'Segunda-feira': 2,
        'Terça-feira': 3,
        'Quarta-feira': 4,
        'Quinta-feira': 5,
        'Sexta-feira': 6,
        'Sábado': 7
    }
    return traducao.get(dia, dia)

In [12]:
traduzir_dia_da_semana('Sunday')

Column<'traduzir_dia_da_semana(Sunday)'>

In [13]:
df_estatisticas_teste = carregar_dataframe_prata(spark_session=spark, assunto='assunto_cities_skylines', metrica='estatisticas')
df_estatisticas_teste = transformacao_dataframe(df_estatisticas_teste)
df_estatisticas_teste.sort('ID_VIDEO').show(truncate=False)

+-------------+--------------------+------------------------+---------------+------------+-----------+---------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------------+-------------------+-----------+---------------+-----------------+-------------------+----------------------+---------------------+----------+
|data_extracao|DATA_PUBLICACAO     |ID_CANAL                |NM_CANAL       |ID_CATEGORIA|ID_VIDEO   |TITULO_VIDEO                                                                     |DESCRICAO                                           

<p>Depara Canal Vídeo</p>

In [14]:
def construir_depara_canal(dataframe: DataFrame, assunto: str):
    dataframe.select(
        dataframe.ID_CANAL,
        dataframe.NM_CANAL,
        dataframe.ID_VIDEO,
        dataframe.TITULO_VIDEO
    ).distinct().sort('ID_CANAL').write.\
        parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/assunto_{assunto}/depara_canal.parquet')


<p>O Número de comentários e likes gera mais views? </p>

In [15]:
def gerar_relacao_like_comentario_view_favoritos(dataframe: SparkSession):
    df_comentarios_likes = dataframe.select(
        dataframe.data_hora_extracao,
        dataframe.data_extracao,
        dataframe.ID_CANAL,
        dataframe.NM_CANAL,
        dataframe.ID_VIDEO,
        dataframe.TITULO_VIDEO,
        dataframe.TAGS,
        dataframe.TOTAL_VISUALIZACOES,
        dataframe.TOTAL_LIKES,
        dataframe.TOTAL_FAVORITOS,
        dataframe.TOTAL_COMENTARIOS
    ).filter(
        # (df_estatisticas.data_extracao == '2023-10-27') &
        (dataframe.TOTAL_VISUALIZACOES > 0) &
        (dataframe.TOTAL_LIKES > 0) 
    ).sort(f.col('TOTAL_VISUALIZACOES').desc())
    return df_comentarios_likes


<p>ETL PARA OBTER RELAÇÃO LIKES - COMENTÁRIOS - VÍDEOS</p>

In [113]:
lista_assunto = [
        'Power BI',
        'Python AND dados',
        'Cities Skylines',
        'Cities Skylines 2'
    ]
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()
    df_estatisticas_etl = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_{assunto}', 
        metrica='estatisticas')
    
    df_estatisticas_etl = transformacao_dataframe(df_estatisticas_etl)
    criar_particao_ouro(dataframe=df_estatisticas_etl, 
                        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='relacao_viusalicacao_comentario_likes',
        colunas_particao=('data_extracao', 'ID_CANAL', 'ID_VIDEO'),
        nome_arquivo='relacao_viusalicacao_comentario_likes.parquet',
    )
    construir_depara_canal(dataframe=df_estatisticas_etl, assunto=assunto)

ETL PARA  Power BI


ETL PARA  Python AND dados


ETL PARA  Cities Skylines


ETL PARA  Cities Skylines 2


<p>Dia de publicação</p>

In [204]:
df_estatisticas.select('data_extracao').distinct().show()

+-------------+
|data_extracao|
+-------------+
|   2023-10-26|
|   2023-10-27|
|   2023-10-25|
|   2023-10-24|
|   2023-10-23|
|   2023-10-22|
|   2023-10-20|
|   2023-10-21|
|   2023-10-16|
|   2023-10-19|
|   2023-10-15|
+-------------+



<p>Públicações por semana</p>

In [271]:
def obter_total_publicacao_video(dataframe : DataFrame) -> DataFrame:
   dataframe =  dataframe.select(
    'DATA_PUBLICACAO',
    'ID_VIDEO'
    ) \
        .sort('ID_VIDEO') \
        .withColumn(
            'SEMANA_TRADUZIDA', 
            traduzir_dia_da_semana(
                f.date_format('DATA_PUBLICACAO', 'EEEE')
            )
            ).distinct() \
                .groupBy('SEMANA_TRADUZIDA'). \
                    count(). \
                        alias('TOTAL_VIDEOS'). \
                            withColumn('INDICE_SEMANA', indice_semana('SEMANA_TRADUZIDA')) \
                    .sort('INDICE_SEMANA')\
                        .select('SEMANA_TRADUZIDA', f.col('TOTAL_VIDEOS.COUNT').alias('TOTAL_VIDEOS'))       
   return dataframe

In [283]:
obter_total_publicacao_video(df_estatisticas_teste).show()

+----------------+------------+
|SEMANA_TRADUZIDA|TOTAL_VIDEOS|
+----------------+------------+
|         Domingo|          39|
|   Segunda-feira|          43|
|     Terça-feira|          51|
|    Quarta-feira|          41|
|    Quinta-feira|          49|
|     Sexta-feira|          37|
|          Sábado|          26|
+----------------+------------+



<p>ETL Para obter o total de vídeos públicado por semaba</p>

In [278]:
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()
    df_estatisticas_etl = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_{assunto}', 
        metrica='estatisticas'
        )
    df_estatisticas_etl = transformacao_dataframe(df_estatisticas_etl)
    df_estatisticas_etl = obter_total_publicacao_video(df_estatisticas_etl)
    criar_particao_ouro(
        dataframe=df_estatisticas_etl, 
        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='total_video_publicado_semana',
        nome_arquivo='total_video_publicado_semana.parquet',
    )
    
    

ETL PARA  Power BI


ETL PARA  Python AND dados


ETL PARA  Cities Skylines
ETL PARA  Cities Skylines 2


<p>Dia da semana com mais visualização</p>

In [287]:
df_estatisticas_teste.show()

+-------------+--------------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------------------+----------------------+---------------------+----------+
|data_extracao|     DATA_PUBLICACAO|            ID_CANAL|            NM_CANAL|ID_CATEGORIA|   ID_VIDEO|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS| data_hora_extracao|DURACAO_VIDEO_SEGUNDOS|TOTAL_CARACTERE_VIDEO|TOTAL_TAGS|
+-------------+--------------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------------------+----------------------+---------------------+----------+
|   2023-10-27|2023-10-24T00

In [18]:
@f.udf(returnType=t.StringType())
def semana(indice: int) -> str:
    semana = {
        1: 'Domingo',
        2: 'Segunda-feira',
        3: 'Terça-feira',
        4: 'Quarta-feira',
        5: 'Quinta-feira',
        6: 'Sexta-feira',
        7: 'Sábado'
    }
    return semana.get(indice, indice)
    

<p>Vísualizações por semana</p>

In [33]:
df_v_teste = df_estatisticas_teste\
    .select('data_extracao', 'ID_CANAL','ID_VIDEO', 'TOTAL_VISUALIZACOES') \
    .withColumn('INDICE_SEMANA', f.dayofweek('data_extracao'))  \
    .withColumn('SEMANA', semana('INDICE_SEMANA')) \
    .groupBy('SEMANA', 'ID_CANAL', 'ID_VIDEO')\
        .max('TOTAL_VISUALIZACOES')\
        .sort('SEMANA')\
        .withColumnRenamed('max(TOTAL_VISUALIZACOES)', 'MAX_TOTAL_VISUALIZACOES')

df_v_teste.show(truncate=False)

+-------+------------------------+-----------+-----------------------+
|SEMANA |ID_CANAL                |ID_VIDEO   |MAX_TOTAL_VISUALIZACOES|
+-------+------------------------+-----------+-----------------------+
|Domingo|UC1mk6EtfMjxR4eEZ7C43zTQ|6qHqkq9yZpM|255                    |
|Domingo|UCwBN6GQiolq3EkRPYC7OTtA|I8PSpoki4T0|144                    |
|Domingo|UCayiH2Y5JBEOSubqoQhufDA|V1jZ4em2xO0|6078                   |
|Domingo|UCJHBJmrVd7ujf2eCEujukog|mKgATax4ecI|4776                   |
|Domingo|UCkXA2EoaXRFW18vrAE01BpQ|7NWFw5SiqRE|27                     |
|Domingo|UCkXA2EoaXRFW18vrAE01BpQ|IY91TrJIHtE|3                      |
|Domingo|UCcn60fs5EMW8A_k9R-e4gEg|IJ8TKnm8VrE|40                     |
|Domingo|UCT1DmITSbFv3tN5GqalE-mg|NGnk0S5RZKo|2506                   |
|Domingo|UCRt5pfs_8hq3-GUSjULoV9Q|wmWIRWYc53A|270                    |
|Domingo|UCDqzBrUOm7guqqyU8eDackg|GliOd9Xpk94|0                      |
|Domingo|UCElNoZcxhVqDtZBSrdEomnA|t4pfQW9O3Lw|2834                   |
|Domin

In [34]:
df_v_testev2 = df_v_teste.withColumn('ISEMANA', indice_semana('SEMANA')) \
        .groupBy('ISEMANA', 'SEMANA') \
        .sum('MAX_TOTAL_VISUALIZACOES') \
        .sort('ISEMANA') \
    .withColumnRenamed('sum(MAX_TOTAL_VISUALIZACOES)', 'TOTAL_VISUALIZACOES')
df_v_testev2 = df_v_testev2.drop('ISEMANA')
df_v_testev2.show()

+-------------+-------------------+
|       SEMANA|TOTAL_VISUALIZACOES|
+-------------+-------------------+
|      Domingo|             525028|
|Segunda-feira|             670037|
|  Terça-feira|             799358|
| Quarta-feira|             928146|
| Quinta-feira|            1076002|
|  Sexta-feira|            1185093|
|       Sábado|             366474|
+-------------+-------------------+



In [27]:
df_v_testev2.printSchema()

root
 |-- ISEMANA: integer (nullable = true)
 |-- SEMANA: string (nullable = true)
 |-- TOTAL_VISUALIZACOES: long (nullable = true)



In [288]:
df_estatisticas_teste.groupBy('data_extracao').max('TOTAL_LIKES').show()

+-------------+----------------+
|data_extracao|max(TOTAL_LIKES)|
+-------------+----------------+
|   2023-10-26|           13784|
|   2023-10-27|           13988|
|   2023-10-25|           13460|
|   2023-10-24|           13033|
|   2023-10-23|           12605|
|   2023-10-22|           12013|
|   2023-10-20|            8899|
|   2023-10-21|           10993|
|   2023-10-16|            1517|
|   2023-10-19|            4955|
|   2023-10-15|             687|
+-------------+----------------+



<p>Desempenho dos Vídeos</p>